# Библиотеки

In [1]:
!pip install -U transformers
!pip install pyTelegramBotAPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 553.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 35.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 17.9 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import telebot
from huggingface_hub import login
from google.colab import userdata # secrets

In [ ]:
TG_TOKEN = userdata.get('TG_TOKEN')
MAX_TOKENS = 1024
login('HUGGINGFACE_TOKEN')

# Модель Gemma-2-2B-it

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def generate(input_text):
    input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

    outputs = model.generate(**input_ids, max_new_tokens=MAX_TOKENS)

    return tokenizer.decode(outputs[0])

# Telegram-Бот

In [ ]:
bot = telebot.TeleBot(TG_TOKEN)

@bot.message_handler(content_types=['text'])
def chat_bot(message):
    msg = bot.reply_to(message, "<Gemma-2-2B-it>>>> ▋")

    generated_text = generate(message.text).replace('<bos>', '<Gemma-2-2B-it>>>>').replace('<end_of_turn>', ' ▋').replace(message.text, '')

    bot.edit_message_text(generated_text, chat_id=message.chat.id, message_id=msg.message_id)

bot.polling()